<a href="https://colab.research.google.com/github/seungbinahn/START_AI/blob/master/02_Machine_Learning%E2%80%8E/03_2_Model_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Selection 
* 최적의 모델 찾기
* 트레인셋과 테스트셋은 서로 달라야 함

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()

train_data = iris.data
train_label = iris.target

# 트레인 셋으로 테스트도 진행
dt_clf.fit(train_data, train_label)
pred = dt_clf.predict(train_data)

# 1.0 이라는 비이상적 결과 발생
print('predictioin accuracy:',accuracy_score(train_label, pred))

predictioin accuracy: 1.0


## K Fold Cross Validataion
* K겹 교차 검증
* K 개의 데이터 Fold를 생성
* 학습과 검증 평가를 반복 수행
* 각 검증 과정 마다 accuracy(정확성) 생성
* 각 iter의 평균 계산

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.datasets import load_iris
import numpy as np

In [3]:
iris=load_iris()
features = iris.data
label = iris.target

kfold=KFold(n_splits=5) # n_splits가 3이면 accruacy는 0이 됨 - 비균등 데이터

print('Iris Data Set Size:',len(features))

fold_index=0
cv_accuracy=[]

Iris Data Set Size: 150


In [4]:
# train index, test_index 분할
for train_index,test_index in kfold.split(features):
  dt_clf = DecisionTreeClassifier(random_state=156)

  x_train, x_test = features[train_index],features[test_index] # index를 이용하여 분할
  y_train, y_test = label[train_index],label[test_index] # index를 이용하여 분할
  
  dt_clf.fit(x_train, y_train) # 모델 생성
  pred = dt_clf.predict(x_test) # 예측
  
  fold_index +=1 
  
  accuracy = np.round(accuracy_score(y_test,pred),4) # 검증
  
  train_size = len(x_train)
  test_size = len(x_test)
  
  print('\n#{0} fold accyracy: {1}, train_size: {2} , val size: {3}'
        .format(fold_index,accuracy,train_size,test_size))
  print('#{0} val index:\n{1}'.format(fold_index,test_index))
  
  cv_accuracy.append(accuracy)


#1 fold accyracy: 1.0, train_size: 120 , val size: 30
#1 val index:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 fold accyracy: 0.9667, train_size: 120 , val size: 30
#2 val index:
[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 fold accyracy: 0.8667, train_size: 120 , val size: 30
#3 val index:
[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 fold accyracy: 0.9333, train_size: 120 , val size: 30
#4 val index:
[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 fold accyracy: 0.7333, train_size: 120 , val size: 30
#5 val index:
[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]


In [5]:
print(cv_accuracy)
print('## avg val accuracy', np.mean(cv_accuracy))

[1.0, 0.9667, 0.8667, 0.9333, 0.7333]
## avg val accuracy 0.9


* 불균등한 데이터의 경우 오류가 발생할 수 있음

## Stratified K Fold 
* 불균형 분포를 가진 데이터 셋을 위한 방식
* stratifed : 계층화된

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
import pandas as pd

iris=load_iris()

df_iris = pd.DataFrame(data=iris.data,columns=iris.feature_names)
df_iris['label']=iris.target
print(df_iris['label'].value_counts())
print(iris.target) # 분포가 균등하지 않음

2    50
1    50
0    50
Name: label, dtype: int64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [7]:
kfold = KFold(n_splits=3)

fold_index=0

for train_index, test_index in kfold.split(df_iris):
  fold_index +=1

  label_train=df_iris['label'].iloc[train_index] #iloc를 이용하여 행 추출
  label_test=df_iris['label'].iloc[test_index] #iloc를 이용하여 행 추출
  
  print('## Cross validation: {0}'.format(fold_index))
  print('Train label distribution:')
  print(label_train.value_counts(), end='\n\n')
  print('Val label distributioin:')
  print(label_test.value_counts(), end='\n\n')
  # 분포가 균등하지 않아 학습되지 않음

## Cross validation: 1
Train label distribution:
2    50
1    50
Name: label, dtype: int64

Val label distributioin:
0    50
Name: label, dtype: int64

## Cross validation: 2
Train label distribution:
2    50
0    50
Name: label, dtype: int64

Val label distributioin:
1    50
Name: label, dtype: int64

## Cross validation: 3
Train label distribution:
1    50
0    50
Name: label, dtype: int64

Val label distributioin:
2    50
Name: label, dtype: int64



### Stratified K Fold를 이용

In [8]:
from sklearn.model_selection import StratifiedKFold

skf=StratifiedKFold(n_splits=3) # stratified fold 사용

fold_index=0

for train_index, test_index in skf.split(df_iris, df_iris['label']):
  fold_index +=1

  label_train=df_iris['label'].iloc[train_index]
  label_test=df_iris['label'].iloc[test_index]
  
  print('## Cross validation: {0}'.format(fold_index))
  
  print('Train label distribution:')
  print(label_train.value_counts(), end='\n\n') # train set의 구성
  
  print('Val label distributioin:')
  print(label_test.value_counts(), end='\n\n') # test set의 구성


## Cross validation: 1
Train label distribution:
2    34
1    33
0    33
Name: label, dtype: int64

Val label distributioin:
1    17
0    17
2    16
Name: label, dtype: int64

## Cross validation: 2
Train label distribution:
1    34
2    33
0    33
Name: label, dtype: int64

Val label distributioin:
2    17
0    17
1    16
Name: label, dtype: int64

## Cross validation: 3
Train label distribution:
0    34
2    33
1    33
Name: label, dtype: int64

Val label distributioin:
2    17
1    17
0    16
Name: label, dtype: int64



### Stratifed K Fold를 사용한 결과


In [9]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

iris=load_iris()
features = iris.data
label=iris.target

dt_clf=DecisionTreeClassifier(random_state=156)

skfold=StratifiedKFold(n_splits=3)

fold_index=0
lst_accuracy=[]

for train_index,test_index in skfold.split(features,label):
  x_train,x_test = features[train_index],features[test_index]
  y_train,y_test = label[train_index],label[test_index]

  dt_clf.fit(x_train,y_train)
  pred=dt_clf.predict(x_test)

  fold_index+=1
  accuracy = np.round(accuracy_score(y_test,pred),4)

  train_size = x_train.shape[0]
  test_size = x_test.shape[0]
  
  print('\n#{0} Cross val accuracy :{1}, train size: {2}, test size: {3}'.format(fold_index, accuracy, train_size, test_size))
  print('#{0} Val index:{1}'.format(fold_index,test_index))
  
  lst_accuracy.append(accuracy) 
  
print('\n## fold val accuracy:', np.round(lst_accuracy, 4))
print('## avg val accuracy:', np.mean(lst_accuracy))



#1 Cross val accuracy :0.98, train size: 100, test size: 50
#1 Val index:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 Cross val accuracy :0.94, train size: 100, test size: 50
#2 Val index:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 Cross val accuracy :0.98, train size: 100, test size: 50
#3 Val index:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## fold val accuracy: [0.98 0.94 0.98]
## avg val accuracy: 0.9666666666666667


# cross_val_score
* 작업 간편화
* estimator, data, label, 지표 등 변수를 전달하여 모델링 수행
* 단점 : 스코어만 보여줌

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris

iris = load_iris()
data = iris.data
label = iris.target

dt_clf = DecisionTreeClassifier(random_state=156)

In [11]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(estimator=dt_clf,
                         X=data, y=label,
                         scoring='accuracy', # 지표
                         cv=3) # cross_validation = 3 개로 나누겠다 - 내부적으로로 Stratified k fold 사용

print('Fold val accuracy:',np.round(scores, 4))
print('Avg val accuracy:', np.round(np.mean(scores), 4))

Fold val accuracy: [0.98 0.94 0.98]
Avg val accuracy: 0.9667


# GridSearchCV
* 모델의 최적 하이퍼 파라미터 도출 : 시간 절약
* 하이퍼 파라미터 : 엔지니어가 결정해서 세팅하는 변수
* 가중치 : 학습의 대상, 딥러닝 / 머신러닝 학습의 결과  
* param_grid 인자로 전달된 파라미터를 순차적으로 적용하여 학습 및 테스트



In [12]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris=load_iris()

x_train,x_test,y_train,y_test = train_test_split(iris.data,
                                                 iris.target,
                                                 test_size=0.2,
                                                 random_state=121)

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dtree=DecisionTreeClassifier() # 결정트리

parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]} # 파라미터 딕셔너리 생성

grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True) # GridSearchCV 생성, refit = True이면 best estimator를 가져올 수 있음
grid_dtree.fit(x_train, y_train) # 모델 생성

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]},

In [14]:
import pandas as pd

scores_df = pd.DataFrame(grid_dtree.cv_results_)
print(scores_df.shape) # shape
print(scores_df.columns) # columns 
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

(6, 13)
Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_max_depth', 'param_min_samples_split', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score'],
      dtype='object')


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [15]:
print('Optimal parameter:', grid_dtree.best_params_) # 최적 하이퍼 파라미터
print('Max accuracy: {0:.4f}'.format(grid_dtree.best_score_)) # 최고 결과

Optimal parameter: {'max_depth': 3, 'min_samples_split': 2}
Max accuracy: 0.9750


### 모델 추출

In [16]:
# 가장 결과가 좋은 모델 가져오기
estimator = grid_dtree.best_estimator_
pred = estimator.predict(x_test)
accuracy_score(y_test, pred)

0.9666666666666667

In [17]:
print(type(estimator))

<class 'sklearn.tree._classes.DecisionTreeClassifier'>
